#1 Get dependencies and import libraries

In [ ]:
!pip install ultralytics -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.5 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import cv2
import os
from google.colab import drive
import zipfile
import csv
import numpy as np
import glob

#2 Get the pre-trained weights

In [ ]:
#create dir for weight
weight_dir = "/content/weight"
os.makedirs(weight_dir, exist_ok=True)

# weight for ship detection
!wget https://github.com/robmarkcole/kaggle-ships-in-satellite-imagery-with-YOLOv8/raw/main/models/yolov8m_best.pt -O /content/weight/yolov8m_shipdetection.pt

#weight for buildings detection
!wget https://github.com/Almirawong/YOLOv8BuildingDetection/raw/main/best-v13andv16.pt -O /content/weight/buildingsdetection.pt

#weight for airplane detection
!wget https://huggingface.co/iturslab/Efficient-YOLO-RS-Airplane-Detection/resolve/main/training/experiment-12/best.pt -O /content/weight/airplanedetection.pt

--2025-09-27 07:54:42--  https://github.com/robmarkcole/kaggle-ships-in-satellite-imagery-with-YOLOv8/raw/main/models/yolov8m_best.pt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/robmarkcole/kaggle-ships-in-satellite-imagery-with-YOLOv8/main/models/yolov8m_best.pt [following]
--2025-09-27 07:54:42--  https://raw.githubusercontent.com/robmarkcole/kaggle-ships-in-satellite-imagery-with-YOLOv8/main/models/yolov8m_best.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52029920 (50M) [application/octet-stream]
Saving to: ‘/content/weight/yolov8m_shipdetection.pt’

/content/weight/yol 100%[===

#3 Extract images for object detection test

In [ ]:
drive.mount('/content/drive')

#Define paths
zip_path = "/content/drive/MyDrive/Final_project/objectdetectionimages.zip"
extract_dir = "/content"

#Create target directory if it does not exist
os.makedirs(extract_dir, exist_ok=True)

#Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Images extracted to: {extract_dir}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Images extracted to: /content


#4 Execute yolo for object detection

In [ ]:
weight_dir = "/content/weight"
models_dict = {
    1: ("Buildings Detection", f"{weight_dir}/buildingsdetection.pt"),
    2: ("Airplane Detection", f"{weight_dir}/airplanedetection.pt"),
    3: ("Ship Detection", f"{weight_dir}/yolov8m_shipdetection.pt")
}

In [ ]:
def run_detection(image_prefix, choice=1, csv_path="/content/detection_results.csv", save_dir="/content/detection_outputs"):
    """
    Run YOLOv8 detection on all images matching a given prefix.
    Saves summary metrics to CSV and detection images to a folder.

    Args:
        image_prefix (str): Path prefix for images (e.g. '/content/images/building_').
        choice (int): 1=Buildings, 2=Airplane, 3=Ship.
        csv_path (str): Path to save CSV results (appends if exists).
        save_dir (str): Directory to save detection output images.
    """
    if choice not in models_dict:
        print("Invalid choice. Defaulting to Buildings Detection.")
        choice = 1

    model_name, model_path = models_dict[choice]
    print(f"\n Loading {model_name} model")
    model = YOLO(model_path)

    # Create save directory if not exists
    os.makedirs(save_dir, exist_ok=True)

    # Find all png images
    image_files = glob.glob(image_prefix + "*.png")

    if not image_files:
        print(f"No images found for prefix: {image_prefix}")
        return

    file_exists = os.path.isfile(csv_path)
    with open(csv_path, "a", newline="") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["Image", "Model", "Detections", "MeanConf", "MaxConf", "MinConf"])

        for image_path in image_files:
            print(f"\nRunning detection on {image_path}")
            image = cv2.imread(image_path)
            if image is None:
                print(f"Image not found: {image_path}")
                continue

            results = model(image, conf=0.65)

            # Save detection image
            save_path = os.path.join(save_dir, os.path.basename(image_path))
            results[0].save(filename=save_path)

            # Collect confidence scores
            confidences = []
            for box in results[0].boxes:
                conf = float(box.conf[0])
                confidences.append(conf)
                print(f"Class: {int(box.cls[0])}, Confidence: {conf:.2f}")

            # Compute summary
            if confidences:
                mean_conf = np.mean(confidences)
                max_conf = np.max(confidences)
                min_conf = np.min(confidences)
                num_det = len(confidences)

                print("\nDetection Summary")
                print(f"Detections: {num_det}")
                print(f"Mean confidence: {mean_conf:.3f}")
                print(f"Max confidence: {max_conf:.3f}")
                print(f"Min confidence: {min_conf:.3f}")

                writer.writerow([os.path.basename(image_path), model_name, num_det, mean_conf, max_conf, min_conf])
            else:
                print("\nNo detections found.")
                writer.writerow([os.path.basename(image_path), model_name, 0, "NA", "NA", "NA"])


In [ ]:
#run the object detection with a specific image and set choice - 1 is buildings, 2 is airplane and 3 is ship
run_detection("/content/objectdetectionimages/ship_", choice=3)


--- Loading Ship Detection model ---

--- Running detection on /content/objectdetectionimages/ship_swinir.png ---

0: 640x640 26 boats, 36.9ms
Speed: 5.1ms preprocess, 36.9ms inference, 19.4ms postprocess per image at shape (1, 3, 640, 640)
Class: 0, Confidence: 0.90
Class: 0, Confidence: 0.87
Class: 0, Confidence: 0.87
Class: 0, Confidence: 0.85
Class: 0, Confidence: 0.85
Class: 0, Confidence: 0.83
Class: 0, Confidence: 0.83
Class: 0, Confidence: 0.80
Class: 0, Confidence: 0.80
Class: 0, Confidence: 0.79
Class: 0, Confidence: 0.79
Class: 0, Confidence: 0.78
Class: 0, Confidence: 0.78
Class: 0, Confidence: 0.78
Class: 0, Confidence: 0.78
Class: 0, Confidence: 0.78
Class: 0, Confidence: 0.77
Class: 0, Confidence: 0.77
Class: 0, Confidence: 0.76
Class: 0, Confidence: 0.76
Class: 0, Confidence: 0.75
Class: 0, Confidence: 0.74
Class: 0, Confidence: 0.73
Class: 0, Confidence: 0.72
Class: 0, Confidence: 0.71
Class: 0, Confidence: 0.66

--- Detection Summary ---
Detections: 26
Mean confidenc

In [ ]:
import shutil

# Create a zip file of detection_outputs
shutil.make_archive("/content/detection_outputs", 'zip', "/content/detection_outputs")
from google.colab import files
files.download("/content/detection_outputs.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>